# Quality Control Function architecture:

Now that we have preprocessed the data, we need to perform some quality-control steps on our pre-processed data to make sure that, for example, the gender of the sample corresponds to that of the questionnaire, iden- tify if there are repeated samples, etc.

## Remove Unreliable Samples function
  - Remove rows with more than 10% missing variables
  - Perform Outlier Detection
  - Remove corresponding samples in other list entries
    
    Potential outliers can be identified among the BC control beads using several outlier detection methods. In the       case below, a simple outlier detection method is used. The approach was to eliminate the outliers by removing         points that were above (Mean + 2*SD) and any points below (Mean - 2*SD)


In [ ]:
def remove_unreliable_samples(samples,threshold):
        
    # Apply the threshold for the missing data per row
    samples=samples.loc[samples['missing']<threshold]
    cpgs=cpgs.loc[samples.index.intersection(cpgs.index)]
    
    # Apply the outlier detection based on the values of  bc1.grn,bc1.red,bc2
    tech_vars=samples[['bc1.grn','bc1.red','bc2']]
    #samples.set_index('sample.id',inplace=True)
    
    
    mean=[]
    sd=[]
    m_1=[]
    m_2=[]
    
    # set the outlier threshold to be between mean + 2sd and mean -2sd
    for m in range(0,tech_vars.shape[1]):
        
        mean_col= tech_vars.iloc[:,m].mean()
        mean.append(mean_col)
    
        sd_col= tech_vars.iloc[:,m].std()
        sd.append(sd_col)
        
        m_1.append(mean[m] - (2 * sd[m]))
        m_2.append(mean[m] + (2 * sd[m]))    
        
          
    for i in range(0,len(mean)):
    
        tech_vars = tech_vars[tech_vars.iloc[:,i]>m_1[i]]
        tech_vars = tech_vars[tech_vars.iloc[:,i]<m_2[i]]
        
        # Print the indices common to the output of the missing data and the ouput of the tech_vars
        common = samples.index.intersection(tech_vars.index)

        samples=samples.loc[common]
    
    return samples

## Infer sex function
   - Create a column which displays sex info
   - Classify sex using a hard boundary threshold
    
The individual sex can be computed from median methylation levels on the chromosome X and the proportion of          missing data on chromosome Y. These have been computed previously and stored in the samples object. The user is allowed to chose the hard boundaries producing the two classes. A plot of the median X chromosome values vs the missing Y chromosome values are plotted to aid the user to decide ont he hard boundaries. It is expected than on the bottom left corner the cluster is representative of males and the top right cluster to females.

In [ ]:
# Visualisation of the median.chrX and missing.chrY plot
plt.scatter(x=samples['median.chrX'],y=samples['missing.chrY'])

def infer_sex(samples,threshold_chrX=0.37,threshold_chrY=0.39):
    
    # From bibliography set hard boundaries to descriminate between males and females
                      
    samples.loc[(samples['median.chrX'] < threshold_chrX) & (samples['missing.chrY'] < threshold_chrY), 'sex'] = 'M'
    samples.loc[(samples['median.chrX'] > threshold_chrX) & (samples['missing.chrY'] > threshold_chrY), 'sex'] = 'F'
    samples.loc[(samples['median.chrX'] < threshold_chrX) & (samples['missing.chrY'] > threshold_chrY), 'sex'] = np.nan
    samples.loc[(samples['median.chrX'] > threshold_chrX) & (samples['missing.chrY'] < threshold_chrY), 'sex'] = np.nan    
    
    #Count the number of males and females
    num_males=samples.loc[samples.sex == 'M', 'sex'].count()
    num_females=samples.loc[samples.sex == 'F', 'sex'].count()
    print("Number of Males:",num_males)
    print("Number of Females:",num_females)
    
    samples.set_index("sample.id",inplace=True)
    
    return samples

## Alternative sex infer function

A K-means algorithm can also be used to perform sex inferal by classifying the samples in the plot belonging to two different classes and then assigning each class to either female or male depending on where they are situated on the median X vs missing Y plot.

In [ ]:
def k_mean_sex_infer(samples):

    from sklearn import datasets
    from sklearn.cluster import KMeans
    import sklearn.metrics as sm
    
    x=samples[['median.chrX','missing.chrY']]
    
    # K Means Cluster
    model = KMeans(n_clusters=2)
    model.fit(x)
    
    # View the results
    # Set the size of the plot
    plt.figure(figsize=(14,7))
     
    # Create a colormap
    colormap = np.array(['red', 'lime'])
    
    # Plot the Original Classifications
    plt.subplot(1, 2, 1)
    plt.scatter(x['median.chrX'], x['missing.chrY'], s=40)
    plt.title('Real Classification')
     
    # Plot the Models Classifications
    plt.subplot(1, 2, 2)
    plt.scatter(x['median.chrX'], x['missing.chrY'], c=colormap[model.labels_], s=40)
    plt.title('K Mean Classification')

    
    samples['sex_Kmeans']=model.labels_
    
    # 1 is coded as female and 0 as male
    samples.loc[(samples['sex_Kmeans']==1),'sex_Kmeans']='F'
    samples.loc[(samples['sex_Kmeans']==0),'sex_Kmeans']='M'

## Visualisation of snps intensities

The allele at a SNP locus can be inferred from SNP intensities measured on the BeadChip.
The plot shows the snps intensities of different samples

In [4]:
def snps_distribution(snps):
    
    for i in range(0,snps.shape[0]):
        a=snps.iloc[i]
        b=a.index
        snp_vals=a.values


        snp_vals=pd.DataFrame(snp_vals)
        snp_vals['snps_name']=b

        snp_vals.drop(snp_vals.index[0],inplace=True)
        snp_vals.columns=['val','snps_name']
        plt.scatter(snp_vals['snps_name'],snp_vals['val'])

## Replicates Identification function
  - Classify snps using hard borders (user defined)
  - Visualise a heatmap to show the distance matrix of the snps
  - Output replicates
    
The function relied on the comparison of samples' snps intensities and sex information to deduce if they appear       too similar and can thus be classified as replicates

In [5]:
def identify_replicates(snps,threshold,samples):
    
    snps.set_index('Unnamed: 0',inplace=True)
    snps.index.name='sample_id'
    
    snps[snps<=0.2]=0
    snps[snps>=0.8]=2
    snps[(snps>0.2 ) & (snps<0.8)]=1
    
    dist_matrix = np.empty((snps.shape[0], snps.shape[0]))
    dist_matrix[:,:] = np.nan

    for i in range(0,snps.shape[0]):
        for j in range(i+1,snps.shape[0]):
            dist_matrix[j, i] = abs(snps.iloc[i,:]-snps.iloc[j,:]).sum()
            dist_m=pd.DataFrame(dist_matrix)
            dist_m.index=snps.index
            dist_m.columns=snps.index
            
    ax = sns.heatmap(dist_m, annot=True)
    ax.set_title('Distance Matrix for SNPSs')
    
    dist_m=dist_m < threshold
    
    rows=[]
    columns=[]
    
    for i in range(0,dist_m.shape[0]):
        for j in range(0,dist_m.shape[0]):
            
            if dist_m.iloc[i,j] == True:
                
                rows.append(dist_m.index[i])
                columns.append(dist_m.index[j])
                
    
    sex_ident=samples['sex']
    
    for n in range (0,len(rows)):
            
        if sex_ident[rows[n]] == sex_ident[columns[n]]:
                
            print('Replicate detected:',rows[n],columns[n]) 